|<h2>Course:</h2>|<h1><a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">A deep understanding of AI language model mechanisms</a></h1>|
|-|:-:|
|<h2>Part 2:</h2>|<h1>Large language models<h1>|
|<h2>Section:</h2>|<h1>Fine-tune pretrained models<h1>|
|<h2>Lecture:</h2>|<h1><b>CodeChallenge HELPER: Evolution of Alice and Edgar<b></h1>|

<br>

<h5><b>Teacher:</b> Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h5>
<h5><b>Course URL:</b> <a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">udemy.com/course/dullms_x/?couponCode=202508</a></h5>
<i>Using the code without the course may lead to confusion or errors.</i>

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BertModel, BertTokenizer

import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt

import requests

In [ ]:
# -> GPU (note: best to use the A100, or whatever has the highest memory)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Exercise 1: Preparations

In [ ]:
class BertForBinaryClassification(nn.Module):
  def __init__(self, num_labels=2):
    super(BertForBinaryClassification, self).__init__()

    # load the pre-trained BERT model
    self.bert = BertModel.from_pretrained('bert-base-uncased')

    # classification head that converts the 768-d pooled output into 2 final outputs
    self.classifier = nn.Linear
    self.dropout = nn.Dropout() # hard-coded dropout at 10%

    # initialize the weights and biases
    nn.init.xavier_uniform_(
    nn.init.zeros_(


  def forward(self, input_ids, attention_mask=None, token_type_ids=None):

    # forward pass through the downloaded (pretrained) BERT
    outputs = self.bert()

    # extract the pooled output and apply dropout
    pooled_output = self.dropout(

    # final push through the classification layer.
    logits =
    return

In [ ]:
# Now upload the trained model (.pt file). Note that it takes a few mins to upload...

In [ ]:
# create a model instance, then replace the params with those from the trained model
bert = BertForBinaryClassification().to(device)
bert.load_state_dict(torch.load('./bert_classifier_AliceVsEdgar.pt'))

# halve the memory of the classifier by converting to float16 from float32
bert.half()

# and toggle on eval (no training)


# and need the BERT tokenizer
bertTokenizer =

In [ ]:
# Eleuther's tokenizer
eleuTokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125m')

# load in two GPTneo's and push to GPU
modelAlice = AutoModelForCausalLM.from_pretrained('EleutherAI/gpt-neo-125m').to(device)
modelEdgar =

In [ ]:
# Alice in Wonderland
text = requests.get('https://www.gutenberg.org/cache/epub/11/pg11.txt').text
aliceTokens = eleuTokenizer.encode(text,return_tensors='pt')[0]

# Edgar Allen Poe
text = requests.get('https://www.gutenberg.org/cache/epub/2148/pg2148.txt').text
edgarTokens = eleuTokenizer.encode(text,return_tensors='pt')[0]

# Exercise 2: Translate text between Eleuther to BERT

In [ ]:
# issue is that they have different tokenizers, so needs to be translated into text and re-tokenized
startingtext = 'Hello, my name is Mike and I like purple.'

# eleuther's tokens:
eleuToks = eleuTokenizer(

# bert's tokens
bertToks = bertTokenizer(

print(f'Starting text:\n{startingtext}')
print(f'\n\nEleuther tokens:\n{eleuToks}')
print(f"\nDecoded using Eleuther:\n{eleuTokenizer.decode}")
print(f"\nDecoded using BERT:\n{bertTokenizer.decode}")

print(f'\n\nBERT tokens:\n{bertToks}')
print(f"\nDecoded using BERT:\n{}")
print(f"\nDecoded using Eleuther:\n{}")

In [ ]:
# text -> Eleuther tokens -> text -> BERT tokens

# 1) to Eleuther tokens
startingtext = 'Hello, my name is Mike and I like purple.'
eleuToks =

# 2) back to text
eleuReconText =

# 3) then to bert tokens
bertToks =

# 4) show the reconstruction
bertTokenizer.decode

In [ ]:
# translation functions
def bert2eleu(bertToks):
  b =
  e =
  return

def eleu2bert(eleuToks):
  e =
  b =
  return


# test
b2e = bert2eleu(bertToks['input_ids'])
e2b = eleu2bert(b2e)

print(eleuTokenizer.decode(b2e))
print(bertTokenizer.decode(e2b))

# Exercise 3: Have BERT classify Alice/Edgar model outputs

In [ ]:
seq_len    = 128 # max sequence length
batch_size =  64

### Write a function that generates tokens for BERT to classify

In [ ]:
# note: this function creates the batch+labels directly on the GPU

# exclude tokens that BERT will ignore
tokens_to_exclude = [
    eleuTokenizer.encode('\n'),
    eleuTokenizer.encode('\n\n'),
    eleuTokenizer.encode('\r'),
    eleuTokenizer.encode('\t'),
    eleuTokenizer.encode(' '),
    [eleuTokenizer.eos_token_id],
]

def batch_for_bert():

  # batch_size should be even
  half_batch =

  # initialize batch tensor and labels
  batch2classify = torch.zeros
  labels = torch.zeros(

  # create random starting tokens
  randstarts = torch.randint(

  # generate tokens in batch
  outs_alice = modelAlice.generate(
      randstarts,
      min_length  = 4*seq_len,
      max_length  = 4*seq_len,
      do_sample   = True,
      early_stopping = False,
      eos_token_id = None,
      pad_token_id = eleuTokenizer.pad_token_id,
      bad_words_ids = tokens_to_exclude,
      repetition_penalty = 1.3
      )

  outs_edgar = modelEdgar.generate(
      randstarts,
      min_length  = 4*seq_len,
      max_length  = 4*seq_len,
      do_sample   = True,
      early_stopping = False,
      eos_token_id = None,
      pad_token_id = eleuTokenizer.pad_token_id,
      bad_words_ids = tokens_to_exclude,
      repetition_penalty = 1.3
      )

  # fill the batch tensor
  for i in range(half_batch):

    # first 1/2 is Alice-generated tokens
    outA = eleu2bert(outs_alice[i,1:])
    batch2classify[i,:] =
    labels[i] = 0  # Label "0" for Alice (was used in training this BERT model)

    # second 1/2 is Edgar-generated tokens


  # remove the large matrices from memory
  del outs_alice,outs_edgar

  return batch2classify, labels

In [ ]:
# test
batch2classify,labels = batch_for_bert()

print(batch2classify.shape)
print(labels.shape)

In [ ]:
# BERT loss function
bert_loss_fun = nn.CrossEntropyLoss()

# forward pass, get model predictions, and report the loss+accuracy
logits = bert
predLabels = torch.argmax
loss = bert_loss_fun

print('\nPredicted labels:\n',predLabels)
print('Actual labels:\n',labels)

print(f'\nLoss: {loss:.4f}')
print(f'\nAccuracy: {}')

# Exercise 4: Fine-tune the models with BERT classification

In [ ]:
# create optimizers and set training params

# ALICE optimizer
optimizerAlice = torch.optim.AdamW(modelAlice.parameters(), lr=1e-5)

# EDGAR optimizer
optimizerEdgar = torch.


# training parameters
num_samples = 121

In [ ]:
# initialize losses
lossAlice    = np.zeros(num_samples)
lossEdgar    = np.zeros(num_samples)
bertAccuracy = np.zeros(num_samples)

for sampli in range(num_samples):


  ### --- ALICE fine-tuning
  # get a batch of data
  ix = torch.randint(len(aliceTokens)-seq_len,size=(batch_size,))
  X  = aliceTokens[ix[:,None] + torch.arange(seq_len)]

  # zero-out gradients, forward pass, get loss

  outputs = modelAlice

  # backprop and store loss
  .backward()
  .step()
  lossAlice[sampli] =
  ### ---------------------


  ### --- EDGAR fine-tuning



  ### ---------------------

  # update progress display
  if sampli%10==0:

    # have the models generate some text
    batch2classify,labels =

    # have BERT classify
    with torch.no_grad(): # turn off gradient calculations
      logits =  # forward pass
    predLabels =  # get model predictions
    bertAccuracy[sampli] =

    print(f

In [ ]:
# plot the losses
_,axs = plt.subplots(1,2,figsize=(12,3.3))
axs[0].plot(,'k',markersize=8,label='ALICE loss')
axs[0].plot(,'b',markersize=8,label='EDGAR loss')
axs[0].legend()
axs[0].set(xlabel='Training batch',ylabel='Loss',xlim=[-1,num_samples],title='GEN model loss')

# plot BERT's accuracy

axs[1].plot(x4bert,,'ks-',markerfacecolor=[.7,.7,.9],markersize=8)
axs[1].set(ylim=[.2,1.02],xlim=[-5,num_samples+4],
           xlabel='Training batch',ylabel='Classification accuracy',title='BERT classifying gen models')

plt.tight_layout()
plt.show()